In [43]:
import pandas as pd
import API_Request as isoapi

In [49]:
df = request_ISO_genfuelmix_daterange(20241001,20241013)
df

,BeginDate,GenMw,FuelCategoryRollup,FuelCategory,MarginalFlag
0,2024-10-01 00:10:21-04:00,259,Hydro,Hydro,N
1,2024-10-01 00:10:21-04:00,6295,Natural Gas,Natural Gas,Y
2,2024-10-01 00:10:21-04:00,3323,Nuclear,Nuclear,N
3,2024-10-01 00:10:21-04:00,0,Other,Other,N
4,2024-10-01 00:10:21-04:00,28,Renewables,Landfill Gas,N
...,...,...,...,...,...
26466,2024-10-13 17:03:06-04:00,28,Renewables,Landfill Gas,N
26467,2024-10-13 17:03:06-04:00,295,Renewables,Refuse,N
26468,2024-10-13 17:03:06-04:00,24,Renewables,Solar,N
26469,2024-10-13 17:03:06-04:00,25,Renewables,Wind,N


In [50]:
time = df['BeginDate'][0]
time

Timestamp('2024-10-01 00:10:21-0400', tz='UTC-04:00')

In [51]:
type(time)

pandas._libs.tslibs.timestamps.Timestamp